#### Basic CNN

In [ ]:
class SimpleCNN1(nn.Module):
    def __init__(self):
      super(SimpleCNN1, self).__init__()

      self.conv_layers = nn.Sequential(
          nn.Conv2d(3, 32, kernel_size=3, padding = 1),
          nn.BatchNorm2d(32),
          nn.ReLU(),
          nn.MaxPool2d(kernel_size=2, stride=2),

          nn.Conv2d(32, 64, kernel_size=3, padding = 1),
          nn.BatchNorm2d(64),
          nn.ReLU(),
          nn.MaxPool2d(kernel_size=2, stride=2),

          nn.Conv2d(64, 128, kernel_size=3, padding = 1),
          nn.BatchNorm2d(128),
          nn.ReLU(),
          nn.MaxPool2d(kernel_size=2, stride=2),

          nn.Conv2d(128, 256, kernel_size=3, padding = 1),
          nn.BatchNorm2d(256),
          nn.ReLU(),
          nn.MaxPool2d(kernel_size=2, stride=2),
      )

      self.gap = nn.AdaptiveAvgPool2d((1, 1))
      self.fc_layers = nn.Sequential(
          nn.Flatten(),
          nn.Linear(256, 512),
          nn.ReLU(),
          nn.Dropout(0.5),
          nn.Linear(512, 1024),
          nn.ReLU(),
          nn.Dropout(0.5),
          nn.Linear(1024, 10)
      )

    def forward(self, x):
      x = self.conv_layers(x)
      x = self.gap(x)
      x = self.fc_layers(x)
      return x


model = SimpleCNN1()

#### Hybrid CNN-Pretrained Model

In [ ]:
class HybridCNN(nn.Module):
  def __init__(self, num_classes=5):
    super(HybridCNN, self).__init__()

    self.feature_extraction = nn.Sequential(
          nn.Conv2d(3, 32, kernel_size = 5, padding = 2),
          nn.BatchNorm2d(32),
          nn.ReLU(),
          nn.MaxPool2d(kernel_size = 2, stride = 2),

          nn.Conv2d(32, 64, kernel_size = 3, padding = 2),
          nn.BatchNorm2d(64),
          nn.ReLU(),
          nn.MaxPool2d(kernel_size = 2, stride = 2),

          nn.Conv2d(64, 128, kernel_size = 5, padding = 2),
          nn.BatchNorm2d(128),
          nn.ReLU(),
          nn.MaxPool2d(kernel_size = 2, stride = 2),

          nn.Conv2d(128, 256, kernel_size = 5, padding = 2),
          nn.BatchNorm2d(256),
          nn.ReLU(),
          nn.MaxPool2d(kernel_size = 2, stride = 2),
    )

    self. 
    for param in self.global_cnn.parameters():
        param.requires_grad = False

    for param in self.global_cnn.blocks[6].parameters():
        param.requires_grad = True


    in_features = self.global_cnn.get_classifier().in_features
    self.global_cnn.classifier = nn.Identity()
    self.global_pool = nn.AdaptiveAvgPool2d((1, 1))

    self.fusion_head = nn.Sequential(
            nn.Linear(in_features + 256 , 512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, num_classes)
        )

  def forward(self, x):
    local_features = self.feature_extraction(x)
    local_feat = self.global_pool(local_features)
    local_feat = torch.flatten(local_feat, 1)

    global_features = self.global_cnn(x)

    fused = torch.cat((local_feat, global_features), dim=1)

    out = self.fusion_head(fused)
    return out

#### Double Fine Tune Hybrid Single Pretrained

In [ ]:
import torch
import torch.nn as nn
import timm

class HybridFineTune(nn.Module):
    def __init__(self, num_classes, convnext_weight_path=None, freeze_backbone=True, dropout=0.3):
        super(HybridFineTune, self).__init__()

        # === EfficientNet backbone (fine-grained features) ===
        self.eff = timm.create_model('efficientnet_b0', pretrained=False, num_classes=0)
        eff_features = self.eff.num_features

        # === ConvNeXt backbone (general features) ===
        self.convnext = timm.create_model('convnext_tiny', pretrained=False, num_classes=0)
        if convnext_weight_path is not None:
            print(f"Loading ConvNeXt weights from {convnext_weight_path} ...")
            state_dict = torch.load(convnext_weight_path, map_location='cpu')
            self.convnext.load_state_dict(state_dict, strict=False)
        conv_features = self.convnext.num_features

        # === Freeze backbone (optional) ===
        if freeze_backbone:
            for p in self.eff.parameters():
                p.requires_grad = False
            for p in self.convnext.parameters():
                p.requires_grad = False

        # === Fusion layer ===
        total_features = eff_features + conv_features
        self.classifier = nn.Sequential(
            nn.Linear(total_features, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(512, num_classes)
        )

    def forward(self, x):
        eff_out = self.eff(x)
        conv_out = self.convnext(x)

        # Concatenate kedua feature
        combined = torch.cat((eff_out, conv_out), dim=1)
        out = self.classifier(combined)
        return out
